In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter
from datetime import datetime

data = pd.read_excel('/content/Uniply_dataset.xlsx')

required_columns = ['course', 'qualify_course', 'bursary', 'req_aps', 'year']
for col in required_columns:
    if col not in data.columns:
        print(f"Error: '{col}' column is missing in the dataset.")
        exit()

def calculate_aps(grades):
    aps = 0
    for subject, grade in grades.items():
        if 'Life Orientation' not in subject:
            aps += grade
    return aps

def get_qualifying_courses(aps_score):
    qualifying_courses = data[data['req_aps'] <= aps_score]['course'].unique().tolist()
    return qualifying_courses

def recommend_course(aps_score, current_year):
    year_data = data[(data['req_aps'] <= aps_score) & (data['year'] == current_year)]
    popular_courses = [course for course, count in Counter(year_data['course']).most_common(3)]
    return popular_courses

def recommend_bursary(recommended_courses):
    bursary_data = data[data['course'].isin(recommended_courses)]
    common_bursaries = [bursary for bursary, count in Counter(bursary_data['bursary']).most_common(3)]

    if len(common_bursaries) < 3:
        common_bursaries += ["National Merit Bursary", "Academic Excellence Bursary", "Community Leadership Bursary"]
        common_bursaries = common_bursaries[:3]

    return common_bursaries

def ai_assistant():
    try:
        current_year = datetime.now().year
        num_courses = int(input("Enter the number of subjects you completed: "))
        grades = {}

        for i in range(num_courses):
            subject = input(f"Enter the name of subject {i+1}: ")
            grade = int(input(f"Enter the grade for {subject}: "))
            grades[subject] = grade

        aps_score = calculate_aps(grades)

        # qualifying courses based on APS
        qualifying_courses = get_qualifying_courses(aps_score)

        # Recommended courses
        recommended_courses = recommend_course(aps_score, current_year)

        # Recommended bursaries
        recommended_bursaries = recommend_bursary(recommended_courses)

        print(f"\nAPS Score: {aps_score}")

        print("\nCourses you qualify for:")
        if qualifying_courses:
            for course in qualifying_courses:
                print(f"- {course}")
        else:
            print("No courses available for the entered APS score.")

        print("\nRecommended courses based on demand:")
        for course in recommended_courses:
            print(f"- {course}")

        print("\nRecommended bursaries:")
        for bursary in recommended_bursaries:
            print(f"- {bursary}")

        # High-demand course
        most_common_course = Counter(data['course']).most_common(1)[0][0]
        #print(f"High-demand course: {most_common_course}")

        # Model training for course recommendation
        X = data[['req_aps']]
        y_course = data['qualify_course']
        X_train_course, X_test_course, y_train_course, y_test_course = train_test_split(X, y_course, test_size=0.2, random_state=42)

        # Train Random Forest for course recommendation
        rf_course = RandomForestClassifier(random_state=42)
        rf_course.fit(X_train_course, y_train_course)

        y_pred_course = rf_course.predict(X_test_course)
        print(f"Course Recommendation Accuracy: {accuracy_score(y_test_course, y_pred_course)}")
        print(f"F1 Score for Course: {f1_score(y_test_course, y_pred_course, average='weighted')}")

        # Bursary recommendation using Random Forest
        y_bursary = data['bursary']
        X_train_bursary, X_test_bursary, y_train_bursary, y_test_bursary = train_test_split(X, y_bursary, test_size=0.2, random_state=42)

        # Train Random Forest for bursary recommendation
        rf_bursary = RandomForestClassifier(random_state=42)
        rf_bursary.fit(X_train_bursary, y_train_bursary)

        y_pred_bursary = rf_bursary.predict(X_test_bursary)
        print(f"Bursary Recommendation Accuracy: {accuracy_score(y_test_bursary, y_pred_bursary)}")
        print(f"F1 Score for Bursary: {f1_score(y_test_bursary, y_pred_bursary, average='weighted')}")

    except ValueError:
        print("Invalid input. Please enter the correct data format.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the AI assistant
ai_assistant()

Enter the number of subjects you completed: 7
Enter the name of subject 1: mathematics
Enter the grade for mathematics: 3
Enter the name of subject 2: english
Enter the grade for english: 3
Enter the name of subject 3: life science
Enter the grade for life science: 3
Enter the name of subject 4: physical science
Enter the grade for physical science: 3
Enter the name of subject 5: life orientation
Enter the grade for life orientation: 3
Enter the name of subject 6: geography
Enter the grade for geography: 4
Enter the name of subject 7: sepedi
Enter the grade for sepedi: 2

APS Score: 21

Courses you qualify for:
- Fasion (3 years)
- Graphic Design (3 years)
- Food Service Management (3 years)
-  Safety Management (3 years)
- Policing (3 years)
- Financial Information Systems(3 years)
- Cost & Management Accounting(3 years)
- Internal Auditing(3 years)
- Human Resources Management(3 years)
- Logistics & Supply Chain Management(3 years)
- Marketing(3 years)
- Retail Business Management(3 